In [39]:
import re
import ast
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql import types

In [2]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "finalProject"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [2]:
!wc -l data/train.txt

10517429 data/train.txt


In [3]:
!wc -l data/test.txt

6042135 data/test.txt


In [3]:
# train = spark.read.option('header', 'false').csv('data/train.txt', sep='\t')

In [12]:
# train.write.format('parquet').save('data/train.parquet')
# train_parquet = spark.read.parquet('data/train.parquet')

In [13]:
# !du data/train.txt
# !du data/train.parquet

2497316	data/train.txt
662221	data/train.parquet


In [7]:
# %%time
# train.count()

CPU times: user 0 ns, sys: 30 ms, total: 30 ms
Wall time: 34 s


10517430

In [6]:
# %%time
# train_parquet.count()

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 1.36 s


10517430

In [10]:
# test = spark.read.option('header','false').csv('data/test.txt', sep='\t')
# test.write.format('parquet').save('data/test.parquet')
# test_parquet = spark.read.parquet('data/test.parquet')

In [12]:
# %%time
# test_parquet.count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 327 ms


6042135

In [3]:
train = spark.read.parquet('data/train.parquet')

In [11]:
# test = spark.read.parquet('data/test.parquet')

In [ ]:
train.summary()

In [ ]:
train.show()

In [10]:
train.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [4]:
df = spark.read.option('header', 'false').csv('data/toy_train.txt', sep='\t')
df.head()

Row(_c0='0', _c1=None, _c2='1', _c3='12', _c4='1', _c5='62526', _c6=None, _c7=None, _c8='3', _c9=None, _c10=None, _c11=None, _c12=None, _c13='3', _c14='05db9164', _c15='08d6d899', _c16='9143c832', _c17='f56b7dd5', _c18='25c83c98', _c19='7e0ccccf', _c20='89a13b6b', _c21='0b153874', _c22='7cc72ec2', _c23='c5fe5cb9', _c24='fa365cf9', _c25='ae1bb660', _c26='d2420e4c', _c27='b28479f6', _c28='bffbd637', _c29='bad5ee18', _c30='07c540c4', _c31='bbf70d82', _c32=None, _c33=None, _c34='0429f84b', _c35=None, _c36='32c7478e', _c37='c0d61a5c', _c38=None, _c39=None)

In [5]:
%%time
df.count()

CPU times: user 10 ms, sys: 0 ns, total: 10 ms
Wall time: 582 ms


15

In [6]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [54]:
df.columns[:5]

['c1']

In [24]:
df[df.columns[2]]

Column<b'_c2[0]'>

In [26]:
df.select(df.columns[2]).show(5)

+---+
|_c2|
+---+
|  1|
|  1|
|  0|
| 23|
|  7|
+---+
only showing top 5 rows



In [27]:
type(df)

pyspark.sql.dataframe.DataFrame

In [30]:
# df.write.format('parquet').save('data/toy_train.parquet')
df = spark.read.parquet('data/toy_train.parquet')
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [37]:
for c in df.columns:
    df = df.withColumnRenamed(c, c.strip('_'))
df.columns[:5]

['c0', 'c1', 'c2', 'c3', 'c4']

In [ ]:
df = df.withColumn(df.columns[1:14]).cast(types.FloatType())
df.printSchema()

In [ ]:
df.select(df.columns[1:14].cast('float'))